In [ ]:
import numpy as np
from scipy.stats import mode
import sys
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
from scipy.io import loadmat
import time
import pandas as pd
import os

# ### 1. Read csv data
# df = pd.read_csv('streetscore/nyc.csv', encoding='utf-8', engine='python')
# df.head(2)

### read downloaded data

In [ ]:
#read downloaded files
DownloadPath = '/Users/hemingyi/Dropbox/MLCProjectData/CrimeImage/'         
dowonloaded_file = os.listdir(DownloadPath)              
# fopen = open(DownloadPath+'/name.txt', 'w')
downloaded = set()
for d in dowonloaded_file: 
    a,b = d.split('.',1)
#     a = a.replace('Front','')
#     a = a.replace('Left','')
#     a = a.replace('Right','')
#     string = a + '\n'  
#     fopen.write(string)
    downloaded.add(a)
# fopen.close()
print ('done')
print(downloaded)

In [ ]:
path = '/Users/hemingyi/Dropbox/MLCProjectData/AggregatedData/'
filename = 'Street_related_crime.csv'

In [ ]:
### Request metaData and download panorama image with correct angle of view### My Google API keys
key = ****************************
import urllib,xmltodict,time,os,os.path
import pandas as pd


df = pd.read_csv(path+filename, encoding='utf-8', engine='python')
print("***Try city %s with %s photos"%(filename,len(df)))
print(df.head(1))
path = 'CaseStudy/origin/'+filename         

atp=0
for i in range(len(df))[0:]:
    FID=i
    ### 1.0 Location information
    lat,lon=(str(df['Latitude'][i]),str(df['Longitude'][i]))
    if FID in downloaded:
        print('Photo %s already exists, skip.'%(FID))
        SkipList.append(FID)
    else:
        #print('Not in downloaded_jpg, try to access it')
    ### 1.1 Generate the urlMetaData for SVI metadata at location (lat,lon)
        urlMetaData='http://maps.google.com/cbk?output=xml&ll=%s,%s'%(lat,lon)
        ### 1.2 Reguest and save the metadata
        try:
            metaDataxml = urllib.request.urlopen(urlMetaData)
            metaData = metaDataxml.read()
            data=xmltodict.parse(metaData)
            ### 1.3 Get Pano Yaw Degree
            if data['panorama']==None:
                SkipList.append(FID)
                print("No panorama data, skip %s"%(FID))
                continue
            else:
                pano_yaw_degree = list(data['panorama']['projection_properties'].items())[1][1]
                #print(data['panorama'])
                ### 2. Request the panorama photo with appropriate pano_yaw_degree
                # 2.1 Set up image size, fov, pitch, heading values
                x,y=('600','300')
                base  = "https://maps.googleapis.com/maps/api/streetview?size=%sx%s&location="%(x,y)
                fov   = '120' # focus of view
                pitch = '0' # Vertical camera view angle towards road
                heading = pano_yaw_degree
                source='outdoor'
                # 2.2 Generate a urlSVI for the StreetViewImage SVI
                urlSVI = base+lat+','+lon+'&key='+key+'&fov='+fov+'&pitch='+pitch+'&heading='+heading+'&source='+source 
                #print(urlSVI)
                # 2.3 Save the SVI image
                filename=str(FID)+'Front.jpg'
                #print(filename)
                urllib.request.urlretrieve(urlSVI, os.path.join(DownloadPath,filename))
                leftHeading = float(heading) - 90
                if leftHeading < 0:
                    leftHeading = leftHeading +360
                urlSVILeft = base+lat+','+lon+'&key='+key+'&fov='+fov+'&pitch='+pitch+'&heading='+str(leftHeading)+'&source='+source 
                filename=str(FID)+'Left.jpg'
                #print(filename)
                urllib.request.urlretrieve(urlSVILeft, os.path.join(DownloadPath,filename))
                rightHeading = float(heading) + 90
                if rightHeading > 360:
                    rightHeading = rightHeading - 360
                urlSVIRight = base+lat+','+lon+'&key='+key+'&fov='+fov+'&pitch='+pitch+'&heading='+str(rightHeading)+'&source='+source 
                filename=str(FID)+'Right.jpg'
                #print(filename)
                urllib.request.urlretrieve(urlSVIRight, os.path.join(DownloadPath,filename))
                atp+=1
                print("Attempt:%s, download photo%s"%(atp,FID))
        except:
            continue
print('Done saving the photo')    